In [34]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import random
import json
import re
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('max_colwidth',50)
import torch
from collections import Counter

from loguru import logger

# 数据的预处理

In [2]:
fname = 'data/processed/train.jsonl'

In [3]:
d = list(json.loads(x) for x in open(fname, 'r', encoding='utf-8').read().strip().split('\n'))
df = pd.DataFrame(d)

## 寻找共现标签并删除

### 寻找共现标签

##### 读取

In [4]:
d = list(json.loads(x) for x in open(fname, 'r', encoding='utf-8').read().strip().split('\n'))
df = pd.DataFrame(d)

In [5]:
b = df['label'].apply(lambda x: 'label_123' in x)

In [6]:
b.to_numpy().astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [7]:
# 获取所有label的set
label_set = set()
for e in df['label']:
    for e_label in e: label_set.add(e_label)

# 每一个label在句子中出现的布尔向量
# 将布尔向量改为10，作为二进制数转化为int，作为该label的fingerprint
s = df.shape[0]
label2binary = {}  # label -> fingerprint
fp2label = {}
for e in tqdm(label_set):
    pos_bool = df['label'].apply(lambda x: e in x).to_numpy().astype(int).tolist()
    fingerprint = int(''.join(list(str(x) for x in pos_bool)))
    if fingerprint not in fp2label:
        fp2label[fingerprint] = [e]
    else:
        fp2label[fingerprint].append(e)

# 找到fingerprint相同的label
same_groups = []
for v in fp2label.values():
    if len(v) >= 2:
        same_groups.append(v)

100%|██████████| 1399/1399 [00:49<00:00, 28.47it/s]


In [8]:
label2equivalent = {}
for e in same_groups:
    label2equivalent[e[0]] = e[1:]

In [9]:
json.dump(same_groups, open('temp_data/equivalent_labels.json', 'w', encoding='utf-8'))
json.dump(label2equivalent, open('temp_data/label2equivalent.json', 'w', encoding='utf-8'))

### 读取共现标签

In [4]:
same_groups = json.load(open('temp_data/equivalent_labels.json', 'r', encoding='utf-8'))
label2equivalent = json.load(open('temp_data/label2equivalent.json', 'r', encoding='utf-8'))

### 删除df中的一致标签

In [6]:
delete_set = set()
for e in same_groups:
    for d in e[1:]:
        delete_set.add(d)

In [7]:
def update_list(l):
    new_list = []
    for e in l:
        if e not in delete_set:
            new_list.append(e)
    return new_list

In [8]:
filtered_df = df.copy()
filtered_df['label'] = filtered_df['label'].apply(update_list)

In [9]:
filtered_df.to_pickle('temp_data/filtered_df.pkl')

### 读取

In [5]:
filtered_df = pd.read_pickle('temp_data/filtered_df.pkl')

## 删除单一样本

In [10]:
label_df = filtered_df.explode('label')['label'].value_counts()
single_labels = list(label_df[label_df == 1].index)
temp_df = filtered_df.copy()
temp_df['status'] = temp_df['label'].apply(lambda x: len(x) == 1 and x[0] in single_labels)
filtered_df = filtered_df.drop(temp_df[temp_df['status']].index)
json.dump(single_labels, open('temp_data/single_labels.json', 'w', encoding='utf-8'))

## 删除成员过于相似的标签

### 编辑距离

In [24]:
def edit_distance(string_a, string_b):
    """编辑距离"""
    m = len(string_a)
    n = len(string_b)
    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    # 从空位置变到string_a每个位置的距离
    for col in range(m + 1):
        dp[0][col] = col
    # 从空位置变到string_b 每个位置的距离
    for row in range(n + 1):
        dp[row][0] = row

    # 填表
    for row in range(1, n+1):
        for col in range(1, m+1):
            if string_a[col-1] != string_b[row-1]:
                dp[row][col] = min(dp[row - 1][col], dp[row - 1][col-1], dp[row][col-1]) + 1
            else:
                dp[row][col] = dp[row-1][col-1]
    return dp[n][m]

def similarity(sentences, max_size = 100):
    l = len(sentences)
    if l > max_size:
        idxes = random.sample(list(range(l)), max_size)
    else:
        idxes = list(range(l))
    scores = []
    for i in tqdm(range(len(idxes))):
        for j in range(i, len(idxes)):
            ed = edit_distance(sentences[idxes[i]], sentences[idxes[j]])
            ed_score = ed / ((len(sentences[idxes[i]]) + len(sentences[idxes[j]]) / 2))
            scores.append(ed_score)
    return sum(scores) / len(scores)

In [25]:
sim_df = filtered_df.explode('label').groupby('label').apply(lambda x: similarity(list(x['text'])))

100%|██████████| 20/20 [00:00<00:00, 53.20it/s]


In [26]:
sim_df.to_pickle('temp_data/editdis_sim_df.pkl')

In [68]:
sim_df.describe()

count    1018.000000
mean        0.560279
std         0.184925
min         0.003132
25%         0.497876
50%         0.601706
75%         0.681853
max         0.861278
dtype: float64

In [73]:
sim_df[sim_df > 0.2].sort_values()[:40]

label
label_1301035    0.201597
label_1483512    0.202544
label_569575     0.202695
label_1254705    0.205452
label_1539842    0.210557
label_728840     0.211910
label_861376     0.211946
label_1361222    0.214728
label_469394     0.217275
label_1228277    0.219119
label_1254785    0.219914
label_809381     0.223557
label_1005765    0.223951
label_739588     0.224429
label_1126058    0.228106
label_599083     0.228289
label_692266     0.228575
label_1496608    0.230246
label_445042     0.234402
label_925279     0.236023
label_409686     0.250038
label_1377375    0.252269
label_714705     0.255294
label_93446      0.258135
label_1034900    0.259766
label_951491     0.261364
label_1282349    0.264283
label_95927      0.265233
label_267825     0.274937
label_185470     0.275262
label_501664     0.276468
label_681693     0.277171
label_1338971    0.279295
label_484108     0.282026
label_469203     0.283646
label_691101     0.287370
label_141255     0.293833
label_1500780    0.296299
label_

In [72]:
(sim_df < 0.8).sum()

991

### BLEU

### 删除

In [ ]:
sim_df

## 包含“超话”的句子实验

In [55]:
filtered_df = pd.read_pickle('temp_data/filtered_df.pkl')

In [51]:
def get_topic(sent):
    splits = [' ', '\t', ',', '.', '，', '。', '！', '？', '?', '!', ';', '；']
    words = re.split(f'[{"|".join(splits)}]', sent)
    for e in words:
        ee = e.split('超话')
        if len(ee) > 1:
            return ee[0]
    return ' '

In [56]:
filtered_df['super_topic'] = filtered_df['text'].apply(get_topic)

In [63]:
filtered_df['super_topic'].value_counts().describe()

count      952.000000
mean        81.212185
std       2053.430740
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max      62937.000000
Name: super_topic, dtype: float64

## 分词

### 直接分词

In [14]:
import jieba

In [15]:
df_splitted = filtered_df.copy()
df_splitted['splitted'] = df_splitted['text'].apply(lambda x: list(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/v2/_14t7h052kgc9sf4crk2qb000000gn/T/jieba.cache
Loading model cost 0.403 seconds.
Prefix dict has been built successfully.


In [16]:
df_splitted.to_pickle('temp_data/df_splitted.pkl')

### 构造词表

In [17]:
vocabulary = set()
for e in tqdm(df_splitted['splitted']):
    vocabulary = vocabulary.union(set(e))
vocabulary = sorted(list(vocabulary))
vocabulary_idx = {x: i for i, x in enumerate(vocabulary)}
json.dump(vocabulary, open('temp_data/vocabulary.json', 'w', encoding='utf-8'))
json.dump(vocabulary_idx, open('temp_data/vocabulary_idx.json', 'w', encoding='utf-8'))

100%|██████████| 77314/77314 [01:23<00:00, 927.66it/s] 


In [33]:
vocabulary = json.load(open('temp_data/vocabulary.json', 'r', encoding='utf-8'))
vocabulary_idx = json.load(open('temp_data/vocabulary_idx.json', 'r', encoding='utf-8'))

### 构造词语在句子中的出现矩阵

In [18]:
df_splitted['splitted'][2]

['这个',
 '七星',
 '连珠',
 '的',
 '现象',
 '居然',
 '真的',
 '存在',
 '，',
 '我',
 '只',
 '在',
 '小说',
 '上面',
 '看过',
 '，',
 '太',
 '神奇',
 '了',
 '吧',
 '，',
 '至于',
 '其他',
 '的',
 '我',
 '也',
 '不',
 '太',
 '懂',
 '，',
 '科学',
 '现象',
 '还是',
 '需要',
 '很多',
 '依据',
 '证明',
 '的']

In [19]:
# shape=(vocab, sentence cnt)，词语在句子中的出现矩阵
vocab_appear = np.zeros((len(vocabulary), df_splitted.shape[0]), dtype=bool)
for i, row in tqdm(enumerate(df_splitted['splitted'])):
    for w in row:
        w_idx = vocabulary_idx[w]
        vocab_appear[w_idx][i] = True
csr_vocab_appear = csr_matrix(vocab_appear)

77314it [00:01, 54538.05it/s]


In [20]:
np.save('temp_data/csr_vocab_appear', csr_vocab_appear)

# 关键词方案

In [56]:
csr_vocab_appear = np.load('temp_data/csr_vocab_appear.npy', allow_pickle=True)
vocab_appear = csr_vocab_appear.todense()
df_splitted = pd.read_pickle('temp_data/df_splitted.pkl')

ValueError: Object arrays cannot be loaded when allow_pickle=False

## 构造标签矩阵

### 标签表

In [21]:
label_vocabulary = []
for e in tqdm(df_splitted['label']):
    label_vocabulary.extend(e)
label_count = sorted(list(Counter(label_vocabulary).items()))
label_vocabulary = list(x[0] for x in label_count)
label_count = list(x[1] for x in label_count)
label_vocabulary_idx = {x: i for i, x in enumerate(label_vocabulary)}

json.dump(label_vocabulary, open('temp_data/label_vocabulary.json', 'w', encoding='utf-8'))
json.dump(label_vocabulary_idx, open('temp_data/label_vocabulary_idx.json', 'w', encoding='utf-8'))
json.dump(label_count, open('temp_data/label_count.json', 'w', encoding='utf-8'))

100%|██████████| 77314/77314 [00:00<00:00, 1844062.66it/s]


### 标签在句子中的出现

In [22]:
# shape=(label_vocab, sentence cnt) 标签在句子中的出现矩阵
label_appear = np.zeros((len(label_vocabulary), df_splitted.shape[0]), dtype=bool)
for i, row in tqdm(enumerate(df_splitted['label'])):
    for l in row:
        l_idx = label_vocabulary_idx[l]
        label_appear[l_idx][i] = True
csr_label_appear = csr_matrix(label_appear)

77314it [00:00, 1462927.76it/s]


In [23]:
np.save('temp_data/csr_label_appear', csr_label_appear)

In [24]:
vocab_appear.shape, label_appear.shape

((87875, 77314), (1023, 77314))

### 验证标签矩阵和词矩阵的正确和一致性

In [25]:
vocab_info = np.where(vocab_appear)
v_idx, s_idx = vocab_info[0], vocab_info[1]
for elem_vidx, elem_sidx in tqdm(list(zip(v_idx, s_idx))):
    cur_word = vocabulary[elem_vidx]
    assert cur_word in df_splitted['splitted'][elem_sidx]

100%|██████████| 2012049/2012049 [00:08<00:00, 249823.71it/s]


In [26]:
label_info = np.where(label_appear)
l_idx, s_idx = label_info[0], label_info[1]
for elem_lidx, elem_sidx in tqdm(list(zip(l_idx, s_idx))):
    cur_label = label_vocabulary[elem_lidx]
    assert cur_label in df_splitted['label'][elem_sidx]

100%|██████████| 97791/97791 [00:00<00:00, 294765.41it/s]


In [27]:
def frag_matmul(a, b, max_dim=100, verbose=False, use_cuda=False):
    # a = (a1, m), b = (m, b2)
    # 将a1和a2化为多个长为100的block
    a_rows, b_columns = [], []
    
    if verbose:
        logger.info('正在碎片化')
    dim_a, dim_b = a.shape[0], b.shape[1]
    a_frag_cnt = (dim_a + max_dim - 1) // max_dim
    b_frag_cnt = (dim_b + max_dim - 1) // max_dim
    for ia in range(a_frag_cnt):
        a_rows.append(a[ia * max_dim: min((ia + 1) * max_dim, a.shape[0])])
    for ib in range(b_frag_cnt):
        b_columns.append(b[:, ib * max_dim: min((ib + 1) * max_dim, b.shape[1])])
    
    results = []
    if verbose:
        logger.info('正在将碎片进行矩阵乘法')
    if use_cuda:
        for i_r in tqdm(range(a_frag_cnt)):
            results.append([])
            for i_c in range(b_frag_cnt):
                cur_a_row = torch.tensor(a_rows[i_r]).to(torch.float).to('cuda')
                cur_b_column = torch.tensor(b_columns[i_c]).to(torch.float).to('cuda')
                res = torch.matmul(cur_a_row, cur_b_column)
                results[-1].append(np.array(res.cpu(), dtype=int))
    else:
        for i_r in tqdm(range(a_frag_cnt)):
            results.append([])
            for i_c in range(b_frag_cnt):
                results[-1].append(np.matmul(a_rows[i_r], b_columns[i_c]))
    
    
    if verbose: logger.info('正在合并碎片')
    np_rows = []
    for erow in tqdm(results):
        np_rows.append(np.concatenate(erow, axis=1))
    np_result = np.concatenate(np_rows)
    return np_result
    

## 根据词语和标签的共现矩阵来找出关键词

In [31]:
word_in = np.load('temp_data/word_in_label.npy')
word_not = np.load('temp_data/word_not_in_label.npy')

In [32]:
word_in_imp = word_in / np.expand_dims(label_count, 0)

In [34]:
temp = (77314 - np.array(label_count)) / len(label_count)
word_not_imp = word_not / np.expand_dims(temp, 0)

In [68]:
word_imp = (word_in_imp > 0.99) & (word_not < 9)
word_imp.sum()

352

In [69]:
poses = np.where(word_imp)
words = []
for x, y in zip(poses[0], poses[1]):
    in_imp, not_imp = word_in[x][y], word_not[x][y]
    cur_word, cur_label = vocabulary[x], label_vocabulary[y]
    words.append({
        'in_imp': in_imp,
        'not_imp': not_imp,
        'cur_word': cur_word,
        'cur_label': cur_label
    })
w = pd.DataFrame(words)

In [71]:
w.sort_values(by='in_imp')

,in_imp,not_imp,cur_word,cur_label
0,1,1,,label_1019030
210,1,1,好,label_1467721
211,1,1,好看,label_1208167
215,1,1,岁,label_1249672
218,1,1,建咯,label_177089
...,...,...,...,...
204,8,8,均,label_808482
88,8,8,...,label_808482
206,8,8,型,label_808482
84,8,8,,label_946002


In [60]:
label_vocabulary[134]

'label_1191241'

In [79]:
(df['text'].apply(len) > 128).sum()

2946